In [1]:
import numpy as np
import pandas as pd

In [3]:
# Company ShopX files read 

In [5]:
df_order = pd.read_excel("Company ShopX - Order Report.xlsx")
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Order ID       400 non-null    int64  
 1   Product Code   400 non-null    object 
 2   Units Ordered  400 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.5+ KB


In [7]:
df_order.head()

,Order ID,Product Code,Units Ordered
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [9]:
df_product = pd.read_excel("Company ShopX - Product Weight.xlsx")
df_product.head()

,Product Code,Product Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [11]:
df_zone = pd.read_excel("Company ShopX - Warehouse&Customer Pin Code and Zone details.xlsx")
df_zone.head()

,Store House Pincode,Customer Area Code,Delivery Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [13]:
# Courier Company Files read

In [15]:
df_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
df_invoice.head()

,AWB Code (Airway Bill Number),Order ID,Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone,Freight Type,Total Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [17]:
df_rates = pd.read_excel("Courier Company - Rates.xlsx")
df_rates.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [19]:
# Company ShopX - Order Report and product report merge

In [21]:
df_merge = pd.merge(df_order,df_product,how="left",on="Product Code")
df_merge.head()

,Order ID,Product Code,Units Ordered,Product Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [23]:
df_merge["Weight_by_company (g)"]= df_merge["Units Ordered"]* df_merge["Product Weight (g)"]
df_merge.head()

,Order ID,Product Code,Units Ordered,Product Weight (g),Weight_by_company (g)
0,2001827036,8904223818706,1.0,127,127.0
1,2001827036,8904223819093,1.0,150,150.0
2,2001827036,8904223819109,1.0,100,100.0
3,2001827036,8904223818430,1.0,165,165.0
4,2001827036,8904223819277,1.0,350,350.0


In [25]:
df_company = df_merge.groupby(["Order ID"])["Weight_by_company (g)"].sum().reset_index()
df_company.head()

,Order ID,Weight_by_company (g)
0,2001806210,220.0
1,2001806226,480.0
2,2001806229,500.0
3,2001806232,1302.0
4,2001806233,245.0


In [27]:
# Merge files  on Order Id Based Company Shopx and Courier Company 

In [29]:
df_all = pd.merge(df_company,df_invoice,how="left",on="Order ID")
df_all.head()

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone,Freight Type,Total Amount (Rs.)
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,500.0,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1302.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,245.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3


In [31]:
df_zone.columns

Index(['Store House Pincode', 'Customer Area Code', 'Delivery Zone'], dtype='object')

In [33]:
df_all = pd.merge(df_all,df_zone[['Customer Area Code', 'Delivery Zone']],on="Customer Area Code",how="left",suffixes=("_by_courier","_by_company"))
df_all

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d
2,2001806229,500.0,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d
3,2001806232,1302.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d
4,2001806233,245.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,b
...,...,...,...,...,...,...,...,...,...,...
169,2001821995,477.0,1091121183730,0.50,121003,342008,d,Forward charges,45.4,b
170,2001822466,1376.0,1091121305541,1.10,121003,342301,d,Forward charges,135.0,b
171,2001823564,672.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,d
172,2001825261,1557.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,d


In [35]:
# Weight slab According to company and courier

In [37]:
df_all["multiplier_company"] = np.ceil(df_all["Weight_by_company (g)"]/500)
df_all["multiplier_courier"] = np.ceil(df_all["Chargeable Weight"]/0.5)

In [39]:
df_all["slab_by_company"] = np.ceil(df_all["Weight_by_company (g)"]/500)*0.5
df_all["slab_by_courier"] = np.ceil(df_all["Chargeable Weight"]/0.5)*0.5
df_all.head()

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company,multiplier_company,multiplier_courier,slab_by_company,slab_by_courier
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,1.0,6.0,0.5,3.0
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,1.0,2.0,0.5,1.0
2,2001806229,500.0,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d,1.0,2.0,0.5,1.0
3,2001806232,1302.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d,3.0,3.0,1.5,1.5
4,2001806233,245.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,b,1.0,2.0,0.5,1.0


In [41]:
# rates findout Delivery Zone Charge type (fwd fixed , fwd additional) Company ShopX and Courier Company

In [43]:
df_rates = df_rates.T.reset_index().rename(columns = {"index":"types",0:"charges"})
df_rates

,types,charges
0,fwd_a_fixed,29.5
1,fwd_a_additional,23.6
2,fwd_b_fixed,33.0
3,fwd_b_additional,28.3
4,fwd_c_fixed,40.1
5,fwd_c_additional,38.9
6,fwd_d_fixed,45.4
7,fwd_d_additional,44.8
8,fwd_e_fixed,56.6
9,fwd_e_additional,55.5


In [45]:
df_rates["del_zone"] = df_rates["types"].apply(lambda s : s.split("_")[1])

In [47]:
df_rates["charge_type"] = df_rates["types"].apply(lambda s : s.split("_")[0]+"_"+s.split("_")[2])

In [49]:
df_rates.head()

,types,charges,del_zone,charge_type
0,fwd_a_fixed,29.5,a,fwd_fixed
1,fwd_a_additional,23.6,a,fwd_additional
2,fwd_b_fixed,33.0,b,fwd_fixed
3,fwd_b_additional,28.3,b,fwd_additional
4,fwd_c_fixed,40.1,c,fwd_fixed


In [51]:
df_rate_final = pd.pivot(data=df_rates,index=["del_zone"],columns=["charge_type"],values=["charges"])

In [53]:
df_rate_final = df_rate_final.droplevel(level = None,axis="columns").reset_index()
df_rate_final.head()

charge_type,del_zone,fwd_additional,fwd_fixed,rto_additional,rto_fixed
0,a,23.6,29.5,23.6,13.6
1,b,28.3,33.0,28.3,20.5
2,c,38.9,40.1,38.9,31.9
3,d,44.8,45.4,44.8,41.3
4,e,55.5,56.6,55.5,50.7


In [55]:
df_rate_final.rename(columns={"del_zone": "Delivery Zone_by_company"}, inplace=True)

In [57]:
df_final = df_all.merge(df_rate_final, on="Delivery Zone_by_company", how="left")

In [59]:
df_final.loc[df_final["Freight Type"].str.lower().str.contains("rto"), "rto_mentioned"] = 1

In [61]:
df_final["rto_mentioned"] = df_final["rto_mentioned"].fillna(0)

In [63]:
df_final.head(2)

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company,multiplier_company,multiplier_courier,slab_by_company,slab_by_courier,fwd_additional,fwd_fixed,rto_additional,rto_fixed,rto_mentioned
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,1.0,6.0,0.5,3.0,28.3,33.0,28.3,20.5,0.0
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,1.0,2.0,0.5,1.0,44.8,45.4,44.8,41.3,0.0


In [65]:
df_final["fwd_charge_by_company"] = df_final["fwd_fixed"]+ (df_final["multiplier_company"]-1)*df_final["fwd_additional"]

In [67]:
df_final["rto_charge_by_company"] = (df_final["rto_fixed"]+ (df_final["multiplier_company"]-1)*df_final["rto_additional"])*(df_final["rto_mentioned"])

In [69]:
df_final["total_charge_by_company"] = df_final["fwd_charge_by_company"]+df_final["rto_charge_by_company"] 
df_final.head()

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company,...,slab_by_company,slab_by_courier,fwd_additional,fwd_fixed,rto_additional,rto_fixed,rto_mentioned,fwd_charge_by_company,rto_charge_by_company,total_charge_by_company
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,...,0.5,3.0,28.3,33.0,28.3,20.5,0.0,33.0,0.0,33.0
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,...,0.5,1.0,44.8,45.4,44.8,41.3,0.0,45.4,0.0,45.4
2,2001806229,500.0,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d,...,0.5,1.0,44.8,45.4,44.8,41.3,0.0,45.4,0.0,45.4
3,2001806232,1302.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d,...,1.5,1.5,44.8,45.4,44.8,41.3,0.0,135.0,0.0,135.0
4,2001806233,245.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,b,...,0.5,1.0,28.3,33.0,28.3,20.5,0.0,33.0,0.0,33.0


In [71]:
df_final["variation_in_charge"] = np.where((df_final["Total Amount (Rs.)"]==df_final["total_charge_by_company"]),"Correctly Charged",
                                           np.where(df_final["Total Amount (Rs.)"]<df_final["total_charge_by_company"],"Under Charged","Over Charged"))

In [73]:
df_final.head()

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company,...,slab_by_courier,fwd_additional,fwd_fixed,rto_additional,rto_fixed,rto_mentioned,fwd_charge_by_company,rto_charge_by_company,total_charge_by_company,variation_in_charge
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,...,3.0,28.3,33.0,28.3,20.5,0.0,33.0,0.0,33.0,Over Charged
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,...,1.0,44.8,45.4,44.8,41.3,0.0,45.4,0.0,45.4,Over Charged
2,2001806229,500.0,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d,...,1.0,44.8,45.4,44.8,41.3,0.0,45.4,0.0,45.4,Over Charged
3,2001806232,1302.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d,...,1.5,44.8,45.4,44.8,41.3,0.0,135.0,0.0,135.0,Correctly Charged
4,2001806233,245.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,b,...,1.0,28.3,33.0,28.3,20.5,0.0,33.0,0.0,33.0,Over Charged


In [75]:
df_final['amount_difference'] = df_final['Total Amount (Rs.)'] - df_final['total_charge_by_company']

In [77]:
## Use np.isclose for floating point comparison

In [79]:
df_final["variation_in_charge"] = np.where(
    np.isclose(df_final["Total Amount (Rs.)"], df_final["total_charge_by_company"], atol=0.01),"Correctly Charged",\
    np.where(df_final["Total Amount (Rs.)"] > df_final["total_charge_by_company"],"Over Charged","Under Charged"))

In [81]:
df_final.head(2)

,Order ID,Weight_by_company (g),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),Delivery Zone_by_company,...,fwd_additional,fwd_fixed,rto_additional,rto_fixed,rto_mentioned,fwd_charge_by_company,rto_charge_by_company,total_charge_by_company,variation_in_charge,amount_difference
0,2001806210,220.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,...,28.3,33.0,28.3,20.5,0.0,33.0,0.0,33.0,Over Charged,141.5
1,2001806226,480.0,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,...,44.8,45.4,44.8,41.3,0.0,45.4,0.0,45.4,Over Charged,44.8


In [83]:
# Create Summary Table

In [85]:
summary = df_final.groupby('variation_in_charge').agg(
    Count =('Order ID',"count"),
    Amount =('amount_difference',"sum")
).reset_index()

In [87]:
summary.info()
summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   variation_in_charge  3 non-null      object 
 1   Count                3 non-null      int64  
 2   Amount               3 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 204.0+ bytes


,variation_in_charge,Count,Amount
0,Correctly Charged,24,5.684342e-14
1,Over Charged,124,6.586600e+03
2,Under Charged,26,-7.764000e+02


In [89]:
report_columns = [
    'Order ID',
    'Weight_by_company (g)',
    'Chargeable Weight',
    'slab_by_company',
    'slab_by_courier',
    'Store House Pincode',
    'Customer Area Code',
    'Delivery Zone_by_company',
    'Delivery Zone_by_courier',
    'Freight Type',
    'Total Amount (Rs.)',
    'fwd_charge_by_company',
    'rto_charge_by_company',
    'total_charge_by_company',
    'amount_difference',
    'variation_in_charge'
]

In [91]:
# Convert data into Excel sheets

In [93]:
writer = pd.ExcelWriter("Expected_Results.xlsx", engine='xlsxwriter')

In [95]:
summary.to_excel(writer,sheet_name="Summary",index=False)

In [97]:
df_final[report_columns].to_excel(writer,sheet_name="Calculations",index=False)

In [99]:
writer.close()

In [101]:
# Excel files reads Summary and Calculations

In [103]:
pd.read_excel("Expected_Results.xlsx",sheet_name="Summary")

,variation_in_charge,Count,Amount
0,Correctly Charged,24,5.684342e-14
1,Over Charged,124,6.586600e+03
2,Under Charged,26,-7.764000e+02


In [105]:
pd.read_excel("Expected_Results.xlsx",sheet_name="Calculations")

,Order ID,Weight_by_company (g),Chargeable Weight,slab_by_company,slab_by_courier,Store House Pincode,Customer Area Code,Delivery Zone_by_company,Delivery Zone_by_courier,Freight Type,Total Amount (Rs.),fwd_charge_by_company,rto_charge_by_company,total_charge_by_company,amount_difference,variation_in_charge
0,2001806210,220,2.92,0.5,3.0,121003,140604,b,b,Forward charges,174.5,33.0,0.0,33.0,141.5,Over Charged
1,2001806226,480,0.68,0.5,1.0,121003,723146,d,d,Forward charges,90.2,45.4,0.0,45.4,44.8,Over Charged
2,2001806229,500,0.71,0.5,1.0,121003,421204,d,d,Forward charges,90.2,45.4,0.0,45.4,44.8,Over Charged
3,2001806232,1302,1.30,1.5,1.5,121003,507101,d,d,Forward charges,135.0,135.0,0.0,135.0,0.0,Correctly Charged
4,2001806233,245,0.78,0.5,1.0,121003,263139,b,b,Forward charges,61.3,33.0,0.0,33.0,28.3,Over Charged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001821995,477,0.50,0.5,0.5,121003,342008,b,d,Forward charges,45.4,33.0,0.0,33.0,12.4,Over Charged
170,2001822466,1376,1.10,1.5,1.5,121003,342301,b,d,Forward charges,135.0,89.6,0.0,89.6,45.4,Over Charged
171,2001823564,672,0.70,1.0,1.0,121003,492001,d,d,Forward and RTO charges,172.8,90.2,86.1,176.3,-3.5,Under Charged
172,2001825261,1557,1.60,2.0,2.0,121003,517128,d,d,Forward and RTO charges,345.0,179.8,175.7,355.5,-10.5,Under Charged
